# TFG: Título del TFG

## Hugo López Álvarez

In [5]:
import numpy    
import pandas   
#import wandb
import torch    
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

## Clases

Definición de la clase DatasetTFG que se usará para entrenar al modelo

In [6]:
class DatasetTFG(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

Definición de la clase Modelo
- La capa1 transforma la dimensión de entrada a 64 neuronas
- La capa2 pasa de las 64 neuronas a 1 neurona

In [36]:
class Modelo(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.capa1 = nn.Linear(input_dim, 64)
        self.capa2 =  nn.Linear(64, 1)
        self.sigprueba = nn.Sigmoid() #Se pasa el volor por una sigmoidea
        
    def forward(self,  X):
        X = self.capa1(X)
        X = self.capa2(X)
        #X = self.sigprueba(X)
        return X

## Cargar datos

In [8]:
fileData = pandas.read_csv('../Datasets/modUQ.csv')

### Comprobación de la obtención correcta del csv

In [9]:
fileData.head()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,...,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label,Attack
0,1424242193040,1424242193043,59.166.0.2,4894,149.171.126.3,53,17,5.0,146,2,...,0,0,0,0,0,0,0,0,0,Benign
1,1424242192744,1424242193079,59.166.0.4,52671,149.171.126.6,31992,6,11.0,4704,28,...,0,91,12,19,0,90,12,19,0,Benign
2,1424242190649,1424242193109,59.166.0.0,47290,149.171.126.9,6881,6,37.0,13662,238,...,0,1843,10,119,0,1843,5,88,0,Benign
3,1424242193145,1424242193146,59.166.0.8,43310,149.171.126.7,53,17,5.0,146,2,...,0,0,0,0,0,0,0,0,0,Benign
4,1424242193239,1424242193241,59.166.0.1,45870,149.171.126.1,53,17,5.0,130,2,...,0,0,0,0,0,0,0,0,0,Benign


### Se convierten las columnas no numéricas para poder utilizarlas con pytorch

In [10]:
fileData['IPV4_SRC_ADDR'] = LabelEncoder().fit_transform(fileData['IPV4_SRC_ADDR'])
fileData['IPV4_DST_ADDR'] = LabelEncoder().fit_transform(fileData['IPV4_DST_ADDR'])
fileData['Attack'] = LabelEncoder().fit_transform(fileData['Attack'])


### Se comprueba que los datos se han transformado correctamente

In [11]:
print(fileData.dtypes)

FLOW_START_MILLISECONDS          int64
FLOW_END_MILLISECONDS            int64
IPV4_SRC_ADDR                    int64
L4_SRC_PORT                      int64
IPV4_DST_ADDR                    int64
L4_DST_PORT                      int64
PROTOCOL                         int64
L7_PROTO                       float64
IN_BYTES                         int64
IN_PKTS                          int64
OUT_BYTES                        int64
OUT_PKTS                         int64
TCP_FLAGS                        int64
CLIENT_TCP_FLAGS                 int64
SERVER_TCP_FLAGS                 int64
FLOW_DURATION_MILLISECONDS       int64
DURATION_IN                      int64
DURATION_OUT                     int64
MIN_TTL                          int64
MAX_TTL                          int64
LONGEST_FLOW_PKT                 int64
SHORTEST_FLOW_PKT                int64
MIN_IP_PKT_LEN                   int64
MAX_IP_PKT_LEN                   int64
SRC_TO_DST_SECOND_BYTES        float64
DST_TO_SRC_SECOND_BYTES  

## Se eliminan los datos con valores infinitos

In [12]:
print("¿Existen valores infinitos en X?: ", numpy.isinf(fileData.values).any())
fileData = fileData.replace([numpy.inf, -numpy.inf], numpy.nan).dropna()
print("¿Siguen existiendo valores infinitos en X?: ", numpy.    isinf(fileData.values).any())

¿Existen valores infinitos en X?:  True
¿Siguen existiendo valores infinitos en X?:  False


### Se separan las características (X) de la etiqueta (Y)

In [13]:
X = fileData.drop(columns=['Label', 'Attack', 'IPV4_SRC_ADDR']).values
Y = fileData['Label'].values

## Se separan los datos del entrenamiento de los datos de prueba
El entrenamiento tendrá el 80% de los datos

La prueba tendrá el 20% de los datos

In [14]:
X_entrena, X_prueba, Y_entrana, Y_prueba = train_test_split(
    X, Y, test_size=0.2, random_state=42,  stratify=Y
)

## Se normalizan los datos

In [15]:
escalador = MinMaxScaler(feature_range=(0,1))
X_entrena_normalizado = escalador.fit_transform(X_entrena)

### Se convierten los datos a tensores de Pytorch

In [16]:
X_entrena_tensor = torch.tensor(X_entrena_normalizado, dtype=torch.float32)
Y_entrena_tensor = torch.tensor(Y_entrana, dtype=torch.long)

## Creación del Dataset personalizado

In [17]:
dataset_entrena = DatasetTFG(X_entrena_tensor, Y_entrena_tensor)

## Creación del DataLoader
El número de muestras por lote será de 1024 (batch)

Los datos de cada época se cogerán de forma aleatoria (shuffle)

In [18]:
batchTam = 2000
dataLoader_entrena = DataLoader(dataset_entrena, batch_size=batchTam, shuffle=True)

## Creación del modelo

In [37]:
modelo = Modelo(input_dim=X_entrena_tensor.shape[1])
print(len(Y_entrena_tensor.unique()))

2


## Se configura pérdida y optimizador

In [41]:
#perdida = nn.BCELoss() if len(Y_entrena_tensor.unique()) > 2 else nn.CrossEntropyLoss()
pos_weight = torch.tensor(19.0)
perdida = nn.BCEWithLogitsLoss(pos_weight=pos_weight) #95/5
optimizador = optim.AdamW(modelo.parameters(), lr=0.001)

## Selección del número de épocas

In [21]:
epocas = 10
print("valor de X: ", X_entrena_tensor)

valor de X:  tensor([[9.9379e-01, 9.9380e-01, 9.5982e-01,  ..., 4.6371e-02, 1.2377e-03,
         4.7259e-03],
        [9.8305e-01, 9.8305e-01, 7.8946e-01,  ..., 4.5381e-03, 8.4163e-04,
         1.5753e-03],
        [7.9305e-03, 7.9149e-03, 6.7202e-01,  ..., 1.4852e-03, 5.9409e-04,
         7.0013e-04],
        ...,
        [9.9584e-01, 9.9584e-01, 4.2275e-01,  ..., 1.1882e-03, 2.9704e-04,
         4.5509e-04],
        [9.8827e-01, 9.8827e-01, 7.0037e-01,  ..., 2.6668e-02, 2.9704e-04,
         3.2556e-03],
        [9.8197e-01, 9.8197e-01, 2.3346e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])


## Bucle de entrenamiento o épocas

In [42]:
for epoca in range(epocas):
    for batch_X, batch_Y in dataLoader_entrena:
        optimizador.zero_grad()     #Se resetan lso gradientes acumulados
        salidas = modelo(batch_X)   #Predirecciones
        calPerdida = perdida(salidas, batch_Y.float().unsqueeze(1))  #Se calcula la pérdida
        calPerdida.backward()       #Se calculas los gradientes
        optimizador.step()          #Se actualizan los pasos
    
    print(f'Época: {epoca + 1}, Pérdida: {calPerdida.item():4f}')
        

Época: 1, Pérdida: 0.020396
Época: 2, Pérdida: 0.009877
Época: 3, Pérdida: 0.004355
Época: 4, Pérdida: 0.001147
Época: 5, Pérdida: 0.002756
Época: 6, Pérdida: 0.001659
Época: 7, Pérdida: 0.004422
Época: 8, Pérdida: 0.000846
Época: 9, Pérdida: 0.000706
Época: 10, Pérdida: 0.000514


## Se guarda el modelo en un fichero

In [ ]:
torch.save(modelo.state_dict(), 'modeloSinLabelNiAttackNiIPSRC.pth')

## Se preparan los datos de prueba

In [43]:
X_prueba_normalizado = escalador.fit_transform(X_prueba)

X_prueba_tensor = torch.tensor(X_prueba_normalizado, dtype=torch.float32)
Y_prueba_tensor = torch.tensor(Y_prueba, dtype=torch.long)

dataset_prueba = DatasetTFG(X_prueba_tensor, Y_prueba_tensor)

dataLoader_prueba = DataLoader(dataset_prueba, batch_size=batchTam)


## Se prueba el modelo

### Se pone el modelo en modo evaluación y se inicializan las variables que se utilizarán para la prueba

In [44]:
modelo.eval()
perdida_prueba= 0
correcto = 0
total = 0

### Se ejecutan las pruebas del modelo

In [52]:
with torch.no_grad():
    for batch_X, batch_Y in dataLoader_prueba:
        salidas = modelo(batch_X)
        calPerdida_prueba = perdida(salidas, batch_Y.float().unsqueeze(1))
        perdida_prueba += calPerdida_prueba.item()
        
        _, prediccion = torch.max(salidas.data, 1)
        total += batch_Y.size(0) 
        correcto += (prediccion == batch_Y).sum().item()

### Se calculan las métricas de la prueba

In [53]:
perdida_prueba /= len(dataLoader_prueba)
precision_prueba = 100 * correcto / total
print(f'Pérdida durante la prueba: {perdida_prueba:.4f}, Exactitud: {precision_prueba: .2f}%')

Pérdida durante la prueba: 0.0075, Exactitud:  95.90%


### BCEWithLogitsLoss

Época: 1, Pérdida: 0.020396

Época: 2, Pérdida: 0.009877

Época: 3, Pérdida: 0.004355

Época: 4, Pérdida: 0.001147

Época: 5, Pérdida: 0.002756

Época: 6, Pérdida: 0.001659

Época: 7, Pérdida: 0.004422

Época: 8, Pérdida: 0.000846

Época: 9, Pérdida: 0.000706

Época: 10, Pérdida: 0.000514

Pérdida durante la prueba: 0.0075, Exactitud:  95.90%
